In [1]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40718 sha256=777b9a5b43908094cacfa086e93ba603e94ba64ab7eda6de29ad6dbc0ba26f3c
  Stored in directory: /home/codespace/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [2]:
import pandas as pd
import googlemaps